Wahlpflichtfach Künstliche Intelligenz I: Praktikum

---

# 03 - Kontrollfluss und Objektorientierung

- [Funktionen](#Funktionen)
- [Klassen](#Klassen)
- [Typisierung](#Typisierung)
- [The Python Data Model](#The-Python-Data-Model)
- [Iterables und Iteratoren](#Iterables-und-Iteratoren)
- [Properties](#Properties)
- [Exceptions](#Exceptions)
- [Factory-Methoden](#Factory-Methoden)
- [Generators](#Generators)
- [Kontextmanager (und IO)](#Kontextmanager-(und-IO))
- [Map, Filter & Reduce](#Map,-Filter-&-Reduce)
- [Decorators](#Decorators)
- [Module](#Module)

#### Erzeugen von Objekten

Objekte einer Klasse meineKlasse können durch den Aufruf:

```python
ref=meineKlasse(arg1,arg2,...)
```

erzeugt werden. Die innerhalb der geschweiften Klammern angeführten Argumente werden dem Konstruktor übergeben. Enthält der Konstruktor als Parameter nur self, dann können die geschweiften Klammern nach dem Klassennamen ganz weggelassen werden.

In [ ]:
class Konto:
        angelegteKonten=0

        def __init__(self, inhaber, autorisiert=["Bankangestellter"], startkap=0):
            self.__inhaber = inhaber
            self.__autorisiert = autorisiert
            self.__kontostand = startkap
            Konto.angelegteKonten += 1

        def __del__(self):
            Konto.angelegteKonten -= 1

        def einzahlen(self,betrag):
            if isinstance(betrag, (float, int)) and betrag>0:
                self.__kontostand +=betrag
                print("Neuer Kontostand:    ", self.__kontostand)
            else:
                print("FEHLER: Falsche Betragsangabe")
            return self.__kontostand

        def auszahlen(self, betrag,initiator):
            if not initiator in self.__autorisiert:
                print(f"{initiator} ist nicht berechtigt")
            elif self.__kontostand < betrag:
                print(f"Es befinden sich nur noch {self.__kontostand:10.2f} Euro auf dem Konto")
            else:
                self.__kontostand -= betrag
            return self.__kontostand

        def abfrage(self):
            return self.__kontostand

In [ ]:
kontoSchwarz=Konto("Schwarz",["Schwarz","Bankangestellter","Papa"],10)
print(kontoSchwarz.einzahlen(1499.00))
print(kontoSchwarz.auszahlen(1000, "Freundin"))
print(kontoSchwarz.abfrage())
print(kontoSchwarz.auszahlen(1600, "Papa"))
print(kontoSchwarz.auszahlen(900, "Schwarz"))

print("Anzahl der Konten  :",Konto.angelegteKonten)

kontoWeiss=Konto("Weiss")

print("Anzahl der Konten  :",Konto.angelegteKonten)

print("Kontostand von Schwarz :",kontoSchwarz.abfrage())
print("Kontostand von Weiss :",kontoWeiss.abfrage())

del(kontoWeiss)

print("Anzahl der Konten  :",Konto.angelegteKonten)

In [ ]:
class GiroKonto(Konto):
    pass

In [ ]:
kontoPhilipp = GiroKonto("Philipp", ["Philipp", "Bankangestellter"], 10)
print(kontoPhilipp.einzahlen(1000.00))

## Fortgeschrittenes Python

Bis jetzt haben Sie das **Bread and Butter** der Programmierung in Python kennengelernt. Sie können jetzt mit einfachen Datentypen, grundlegenden Operatoren, Datensammlungen und Kontrollfluss umgehen. Außerdem wissen Sie, wie Sie in Python Schleifen erstellen und Codeabschnitte mit Funktionen wiederverwenden können. In diesem Abschnitt sehen wir uns eine Reihe von Spracheigenschaften an, die Python im Vergleich zu anderen Programmiersprachen besonders machen. 

Alle Datentypen sind Objekte?! Zumindest sind alle eine Instanz von `object`:

In [ ]:
isinstance(2, object), isinstance(2.0, object), isinstance(True, object), 

Objekte haben normalerweise Methoden. Selbst etwas so Einfaches wie ein Boolean hat eine Menge Methoden:

In [ ]:
dir(True)

---

Wahlpflichtfach Künstliche Intelligenz I: Praktikum